In [5]:
import datetime as dt

import requests
import pandas as pd
import numpy as np
import matplotlib
import json    
import requests




In [2]:


def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[3:]


            














In [8]:
resp = requests.get('https://api.kraken.com/0/public/AssetPairs?')
data = resp.json()
def getmarkets(data):
    pairs = []
    navne = list(data["result"].keys())
    for navn in navne:
        if navn[-3:] == "USD":
            pairs.append((navn))
    return pairs


perpnames = getmarkets(data)



In [4]:
def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(hours = lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time

In [5]:
import requests
import pandas as pd

def ohlc(name, tf):
    _start_time, _end_time = now(400) 
    data = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={name}&interval={tf}') 
    data = data.json()
    data = pd.DataFrame(data['result'])
    # Drop the last column
    data = data.iloc[:, :-1]
    
    
    return data

In [1]:
def modify_list(lst):
    return lst[4]

In [7]:
dfs = []
tf = 240

for perp in perpnames: 
    dfs.append(ohlc(perp, tf))

In [4]:
df = pd.concat(dfs, axis=1)
df = df.dropna(axis="columns")
modified_df = df.applymap(modify_list)
modified_df = modified_df.astype(float)
modified_df = modified_df.tail(30)
modified_df.reset_index(drop=True, inplace=True)
#search_result = df.applymap(lambda x: not isinstance(x, list)).any().any()
#search_result1 = df.isin(["NaN"]).any().any()


modified_df

NameError: name 'dfs' is not defined

In [9]:
def divs(closes, columns, ob=50, os=50, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = get_rsi(closes.iloc[:, columns], 14)
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['bullish_rsi_div', 'bearish_rsi_div']]

rsidata = []
cols = len(modified_df.axes[1])
for column in range(cols):
   rsidata.append(divs(modified_df, column))


In [10]:
alle = pd.concat(dict(zip(perpnames,rsidata)), axis=1)

In [11]:
alle

1INCHUSD                         AAVEUSD                  \
   bullish_rsi_div bearish_rsi_div bullish_rsi_div bearish_rsi_div   
17               0               0               0               0   
18               0               0               0               0   
19               0               0               0               0   
20               0               0               0               0   
21               0               0               0               0   
22               0               0               0               0   
23               0               0               0               0   
24               0               0               0               0   
25               0               1               0               0   
26               0               1               0               1   
27               0               1               0               1   
28               0               0               0               0   
29               0               0               0               0   

            ACAUSD                          ACHUSD                  \
   bullish_rsi_div bearish_rsi_div bullish_rsi_div bearish_rsi_div   
17               0               0               0               0   
18               0               0               0               0   
19               0               0               0               0   
20               0               1               0               0   
21               0               0               0               0   
22               0               0               0               1   
23               0               0               0               1   
24               0               0               0               1   
25               0               0               0               0   
26               0               0               0               0   
27               0               0               0               0   
28               0               0               0               0   
29               0               0               0               0   

            ADAUSD                  ...        XETHZUSD                  \
   bullish_rsi_div bearish_rsi_div  ... bullish_rsi_div bearish_rsi_div   
17               0               0  ...               0               0   
18               0               0  ...               0               0   
19               0               0  ...               0               0   
20               0               0  ...               0               0   
21               0               0  ...               0               0   
22               0               0  ...               0               0   
23               0               0  ...               0               0   
24               0               0  ...               0               0   
25               0               0  ...               0               0   
26               0               1  ...               0               1   
27               0               0  ...               0               1   
28               0               0  ...               0               0   
29               0               0  ...               0               0   

          XLTCZUSD                        XMLNZUSD                  \
   bullish_rsi_div bearish_rsi_div bullish_rsi_div bearish_rsi_div   
17               0               0               0               0   
18               0               1               0               0   
19               0               0               0               0   
20               0               0               0               0   
21               0               0               0               0   
22               0               0               0               0   
23               0               0               0               0   
24               0               0               0               0   
25               0               0               0               0   
26          

In [15]:
cols = (alle.iloc[-3:] == 1).any()
websitedata = alle.iloc[-3:][cols[cols].index]

print(websitedata)

           API3USD          ARBUSD        ATLASUSD         ATOMUSD  \
   bearish_rsi_div bearish_rsi_div bearish_rsi_div bearish_rsi_div   
29               1               1               1               1   

           BANDUSD          BLZUSD          BTTUSD         EGLDUSD  \
   bearish_rsi_div bearish_rsi_div bearish_rsi_div bearish_rsi_div   
29               1               1               1               1   

            FLRUSD        FORTHUSD  ...          LCXUSD          LRCUSD  \
   bearish_rsi_div bearish_rsi_div  ... bearish_rsi_div bearish_rsi_div   
29               1               1  ...               1               1   

           MOVRUSD         NANOUSD        POLISUSD          SNXUSD  \
   bearish_rsi_div bearish_rsi_div bearish_rsi_div bearish_rsi_div   
29               1               1               1               1   

            STGUSD          TLMUSD            TUSD        XLTCZUSD  
   bullish_rsi_div bearish_rsi_div bearish_rsi_div bearish_rsi_div  
29 

In [16]:
websitedata


,API3USD,ARBUSD,ATLASUSD,ATOMUSD,BANDUSD,BLZUSD,BTTUSD,EGLDUSD,FLRUSD,FORTHUSD,...,LCXUSD,LRCUSD,MOVRUSD,NANOUSD,POLISUSD,SNXUSD,STGUSD,TLMUSD,TUSD,XLTCZUSD
,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,...,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div,bullish_rsi_div,bearish_rsi_div,bearish_rsi_div,bearish_rsi_div
29,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
